<a href="https://colab.research.google.com/github/andremonroy/stanWeinstein/blob/main/weinstein_entradas%2C_salidas%2C_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Backtesting Estrategia Stan Weinstein (Fase 1 → Fase 2)

Este notebook implementa el framework de *scoring* y backtesting semanal que definiste, usando `pandas`, `numpy`, `matplotlib` y `yfinance`.
Incluye:
- Cálculo de **WMA30** y su **pendiente**
- Detección de **bases/consolidaciones**
- **RS vs benchmark** y pendiente del RS
- **Resistencias** (últimos 2 y 10 años / ATH 10 años)
- Análisis de **volumen** en la base
- Detección de **breakout** con volumen
- Sistema de **scoring (0–10)**
- **Backtest** con reglas de entrada (score ≥ 8) y salida (WMA30 con pendiente negativa)
- **Métricas** (CAGR, Sharpe, % ganadoras, MaxDD, etc.) y gráficos (score, equity curve, ejemplo de base+ruptura).

> **Requisitos**: conexión a Internet para descargar datos con `yfinance`.


In [1]:
#Celda 1
# === Instalación (si hace falta) ===
!pip install yfinance pandas numpy matplotlib scipy tqdm


In [2]:
#Celda 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from tqdm import tqdm
import math

plt.rcParams['figure.figsize'] = (12, 6)
pd.options.display.float_format = '{:,.4f}'.format


## Utilidades

In [3]:
#Celda 3
def to_weekly(df: pd.DataFrame) -> pd.DataFrame:
    """Resample diario → semanal (OHLCV). Semana cierra el viernes."""
    if not df.index.inferred_type == 'datetime64':
        df = df.set_index(pd.to_datetime(df['Date']))
    o = df['Open'].resample('W-FRI').first()
    h = df['High'].resample('W-FRI').max()
    l = df['Low'].resample('W-FRI').min()
    c = df['Close'].resample('W-FRI').last()
    v = df['Volume'].resample('W-FRI').sum()
    out = pd.DataFrame({'Open': o, 'High': h, 'Low': l, 'Close': c, 'Volume': v})
    out.dropna(inplace=True)
    out['Date'] = out.index
    return out

def WMA(series: pd.Series, window: int) -> pd.Series:
    """Weighted Moving Average con pesos lineales 1..window (más peso a lo reciente)."""
    weights = np.arange(1, window+1, dtype=float)
    def f(x):
        return np.dot(x, weights) / weights.sum()
    return series.rolling(window).apply(f, raw=True)

def pct_change_safe(series: pd.Series, periods=1):
    return series.pct_change(periods=periods).replace([np.inf, -np.inf], np.nan)

def rolling_max_drawdown(equity: pd.Series) -> float:
    peak = equity.cummax()
    dd = (equity/peak - 1.0).min()
    return dd

def sharpe_ratio(returns: pd.Series, periods_per_year=52, rf=0.0) -> float:
    excess = returns - rf/periods_per_year
    if excess.std(ddof=0) == 0 or np.isnan(excess.std(ddof=0)):
        return np.nan
    return np.sqrt(periods_per_year) * excess.mean() / excess.std(ddof=0)

def CAGR(equity: pd.Series, periods_per_year=52) -> float:
    if equity.empty:
        return np.nan
    total_return = equity.iloc[-1] / equity.iloc[0] - 1.0
    years = len(equity) / periods_per_year
    if years <= 0:
        return np.nan
    return (1+total_return)**(1/years) - 1

def recent_swing_highs(series: pd.Series, lookback: int = 104, order: int = 3) -> pd.Series:
    """Aproximación: puntos que son máximos locales (swing highs)."""
    # Máximo local simple: mayor que vecinos +/- order
    highs = series.copy()
    mask = pd.Series(False, index=series.index)
    for i in range(order, len(series)-order):
        if series.iloc[i] == series.iloc[i-order:i+order+1].max():
            mask.iloc[i] = True
    return series.where(mask)

def nearest_resistance_above(price: float, swings: pd.Series) -> Optional[float]:
    vals = swings.dropna()
    candidates = vals[vals > price]
    return candidates.min() if not candidates.empty else np.nan

def ath(series: pd.Series, lookback: int) -> float:
    return series.tail(lookback).max()


## Criterios de Score

In [4]:
#Celda 4
@dataclass
class ScoreComponents:
    wma30_slope: int
    base_strength: int
    rs: int
    resistance: int
    volume_pattern: int
    breakout: int
    total: int

def compute_wma30_and_slope(w: pd.DataFrame) -> Tuple[pd.Series, pd.Series, pd.Series]:
    w['WMA30'] = WMA(w['Close'], 30)
    # Pendiente semanal como % (WMA vs WMA -1)
    w['WMA30_slope_pct'] = pct_change_safe(w['WMA30'])
    # Discretización: negativa=0, plana (-0.1%..+0.1%)=2, positiva=1
    cond_neg = w['WMA30_slope_pct'] < -0.001
    cond_flat = (w['WMA30_slope_pct'] >= -0.001) & (w['WMA30_slope_pct'] <= 0.001)
    slope_score = pd.Series(1, index=w.index)
    slope_score[cond_neg] = 0
    slope_score[cond_flat] = 2
    return w['WMA30'], w['WMA30_slope_pct'], slope_score

def detect_current_base(w: pd.DataFrame, max_lookback=60, volat_threshold=0.10) -> Tuple[pd.Series, pd.Series, pd.Series]:
    """Para cada semana t, estima si está en base: rango % en ventana que *termina* en t.
    Devuelve: base_len, base_volatility, base_high (techo).
    """
    base_len = pd.Series(0, index=w.index, dtype=int)
    base_vol = pd.Series(np.nan, index=w.index, dtype=float)
    base_high = pd.Series(np.nan, index=w.index, dtype=float)
    for i in range(len(w)):
        # explorar ventanas que terminen en i, hasta max_lookback
        best_len = 0; best_vol = np.nan; best_high = np.nan
        for L in range(5, min(max_lookback, i)+1):
            win = w.iloc[i-L+1:i+1]
            mx, mn = win['Close'].max(), win['Close'].min()
            mid = (mx+mn)/2.0
            volat = (mx-mn)/mid if mid != 0 else np.nan
            if volat <= volat_threshold:
                best_len = L; best_vol = volat; best_high = mx
        if best_len > 0:
            base_len.iloc[i] = best_len
            base_vol.iloc[i] = best_vol
            base_high.iloc[i] = best_high
    return base_len, base_vol, base_high

def score_base_strength(base_len: pd.Series, base_vol: pd.Series) -> pd.Series:
    s = pd.Series(0, index=base_len.index, dtype=int)
    s[(base_len >= 5) & (base_len < 15)] = 1
    s[(base_len >= 15) & (base_len <= 40) & (base_vol < 0.10)] = 2
    return s

def compute_RS(asset_close: pd.Series, bench_close: pd.Series, lookback_weeks: int = 26) -> Tuple[pd.Series, pd.Series, pd.Series]:
    """RS(t) = retorno log acumulado asset - benchmark en ventana lookback.
    RS_slope: pendiente semanal del RS (dif primera).
    RS score: 0 si cayendo, 2 si RS<0 pero pendiente positiva >=3 semanas, 1 si RS>0 y subiendo.
    """
    asset_ret = np.log(asset_close / asset_close.shift(lookback_weeks))
    bench_ret = np.log(bench_close / bench_close.shift(lookback_weeks))
    RS = asset_ret - bench_ret
    RS_diff = RS.diff()
    # condición "subiendo": RS_diff > 0
    up = RS_diff > 0
    # Pendiente positiva >= 3 semanas consecutivas
    pos3 = up.rolling(3).apply(lambda x: 1 if np.all(x==1) else 0, raw=True).fillna(0) == 1
    score = pd.Series(0, index=RS.index, dtype=int)
    score[(RS < 0) & pos3] = 2
    score[(RS > 0) & up] = 1
    return RS, RS_diff, score

def score_resistance(w: pd.DataFrame, years_2=104, years_10=520, near_ath_pct=0.05) -> pd.Series:
    """Evalúa la distancia a la resistencia más cercana (swing high) por encima del precio actual,
    usando últimos 2 años y 10 años. Si no hay resistencia (porque está cerca de ATH 10 años),
    se considera 'cerca de ATH'.
    Reglas:
      - <10% arriba -> 0
      - 10–20% -> 1
      - >=20% o cerca del ATH(10y) -> 2
    """
    sh_2y = recent_swing_highs(w['High'], lookback=years_2)
    sh_10y = recent_swing_highs(w['High'], lookback=years_10)
    s = pd.Series(0, index=w.index, dtype=int)
    for i in range(len(w)):
        price = w['Close'].iloc[i]
        # Buscar resistencia 2 años (más relevante operativamente)
        r2 = nearest_resistance_above(price, sh_2y.iloc[max(0,i-years_2):i+1])
        # Chequear ATH 10 años
        ath10 = ath(w['High'].iloc[max(0,i-years_10):i+1], lookback=min(years_10, i+1))
        near_ath = not np.isnan(ath10) and (ath10 - price)/ath10 <= near_ath_pct
        if near_ath:
            s.iloc[i] = 2
        else:
            if np.isnan(r2):
                # sin resistencia identificable arriba en 2 años → fallback 10 años
                r10 = nearest_resistance_above(price, sh_10y.iloc[max(0,i-years_10):i+1])
                target = r10
            else:
                target = r2
            if np.isnan(target):
                # tampoco hay por encima → interpretar como cielo despejado
                s.iloc[i] = 2
            else:
                dist = (target/price) - 1.0
                if dist < 0.10:
                    s.iloc[i] = 0
                elif dist < 0.20:
                    s.iloc[i] = 1
                else:
                    s.iloc[i] = 2
    return s

def score_volume_pattern(w: pd.DataFrame, base_len: pd.Series) -> pd.Series:
    """Analiza el volumen durante bases:
    - 2 puntos:
        volumen medio en el último cuarto de la base > 20% al de la primera mitad (expansión al final)
    - 1 punto:
        volumen medio de la base < promedio de 52 semanas (seco/decreciente)
    - 0 puntos: desordenado
    """
    s = pd.Series(0, index=w.index, dtype=int)
    vol52 = w['Volume'].rolling(52).mean()
    for i in range(len(w)):
        L = base_len.iloc[i]
        if L >= 5:
            win = w['Volume'].iloc[i-L+1:i+1]
            half = max(1, L//2)
            qtr = max(1, L//4)
            first_half = win.iloc[:half].mean()
            last_q = win.iloc[-qtr:].mean()
            if first_half > 0 and last_q / first_half >= 1.2:
                s.iloc[i] = 2
            elif win.mean() < vol52.iloc[i]:
                s.iloc[i] = 1
            else:
                s.iloc[i] = 0
    return s

def detect_breakout(w: pd.DataFrame, base_high: pd.Series, vol_mult: float = 2.0, vol_window: int = 10) -> pd.Series:
    """Breakout si Close rompe el techo de la base con Volumen >= vol_mult * promedio(vol_window)."""
    vol_avg = w['Volume'].rolling(vol_window).mean()
    cond = (w['Close'] > base_high) & (w['Volume'] >= vol_mult * vol_avg)
    return cond.fillna(False).astype(int)

def assemble_score(w_slope: pd.Series, base_strength: pd.Series, rs_score: pd.Series,
                   resist_score: pd.Series, vol_score: pd.Series, breakout_flag: pd.Series) -> pd.Series:
    total = w_slope.fillna(0).astype(int) +             base_strength.fillna(0).astype(int) +             rs_score.fillna(0).astype(int) +             resist_score.fillna(0).astype(int) +             vol_score.fillna(0).astype(int) +             (2 * breakout_flag.fillna(0).astype(int))  # Breakout cuenta como 2
    return total.clip(lower=0, upper=10)


## Pipeline por ticker

In [5]:
#Celda 5
def process_ticker_weekly(ticker: str, benchmark: str = 'SPY', start: str = '2010-01-01') -> Dict[str, pd.DataFrame]:
    # Descarga datos (Close ya ajustado con auto_adjust=True)
    data = yf.download(ticker, start=start, auto_adjust=True, progress=False)
    if data.empty:
        raise ValueError(f"Sin datos para {ticker}")
    data = data.reset_index()
    data.rename(columns=lambda s: s.title(), inplace=True)  # estandarizar nombres
    w = to_weekly(data)

    # Benchmark
    bench = yf.download(benchmark, start=start, auto_adjust=True, progress=False)
    if bench.empty:
        raise ValueError(f"Sin datos para benchmark {benchmark}")
    bench = bench.reset_index()
    bench.rename(columns=lambda s: s.title(), inplace=True)
    wb = to_weekly(bench)

    # Alinear por intersección de fechas (más robusto que .loc[w.index])
    w = w.set_index('Date')
    wb = wb.set_index('Date')
    common_idx = w.index.intersection(wb.index)
    if len(common_idx) == 0:
        raise ValueError(f"No hay fechas comunes entre {ticker} y {benchmark}")
    w = w.loc[common_idx].copy()
    wb = wb.loc[common_idx].copy()

    # Indicadores
    WMA30, WMA30_slope_pct, slope_score = compute_wma30_and_slope(w.copy())
    base_len, base_vol, base_high = detect_current_base(w.copy())
    base_score = score_base_strength(base_len, base_vol)
    RS, RS_diff, RS_score = compute_RS(w['Close'], wb['Close'])
    resist_score = score_resistance(w)
    vol_score = score_volume_pattern(w, base_len)
    breakout = detect_breakout(w, base_high)

    score_total = assemble_score(slope_score, base_score, RS_score, resist_score, vol_score, breakout)

    out = w.copy()
    out['WMA30'] = WMA30
    out['WMA30_slope_pct'] = WMA30_slope_pct
    out['BaseLen'] = base_len
    out['BaseVol'] = base_vol
    out['BaseHigh'] = base_high
    out['RS'] = RS
    out['RS_diff'] = RS_diff
    out['RS_score'] = RS_score
    out['Resist_score'] = resist_score
    out['Vol_score'] = vol_score
    out['Breakout'] = breakout
    out['Score'] = score_total
    out['BenchClose'] = wb['Close']  # ya alineado por common_idx

    return {'data': out}


## Backtest (entrada score ≥ 8; salida cuando WMA30 pendiente negativa)

In [6]:
#Celda 6
@dataclass
class Trade:
    entry_date: pd.Timestamp
    entry_price: float
    exit_date: pd.Timestamp
    exit_price: float
    ret: float
    max_dd: float

def backtest_from_scores(df: pd.DataFrame, entry_score: int = 8) -> Tuple[pd.Series, List[Trade]]:
    """Estrategia long-only por ticker:
      - Entrada: semana donde Score cruza desde < entry_score a >= entry_score y Breakout==1
      - Salida: primera semana donde WMA30_slope_pct < -0.001 (pendiente negativa)
      - Capital completamente invertido o en cash (sin piramidación)
    Devuelve curva de equity normalizada a 1.0 y lista de trades.
    """
    close = df['Close']
    slope = df['WMA30_slope_pct']
    score = df['Score']
    breakout = df['Breakout'] == 1

    in_pos = False
    entry_px = None
    entry_idx = None
    equity = pd.Series(index=df.index, dtype=float)
    equity.iloc[0] = 1.0
    trades: List[Trade] = []
    peak_equity = 1.0
    open_date = None

    for i in range(1, len(df)):
        date = df.index[i]
        prev_score = score.iloc[i-1]
        now_score = score.iloc[i]
        # Señal de compra
        buy_signal = (prev_score < entry_score) and (now_score >= entry_score) and breakout.iloc[i]
        sell_signal = (slope.iloc[i] < -0.001)

        if not in_pos and buy_signal:
            in_pos = True
            entry_px = close.iloc[i]
            entry_idx = i
            open_date = date

        if in_pos and sell_signal:
            # cerrar trade
            exit_px = close.iloc[i]
            trade_slice = close.iloc[entry_idx:i+1]
            # max drawdown del trade en términos de precio (no equity)
            peak = trade_slice.cummax()
            dd = (trade_slice/peak - 1.0).min()
            trades.append(Trade(entry_date=open_date, entry_price=entry_px,
                                exit_date=date, exit_price=exit_px,
                                ret=exit_px/entry_px - 1.0, max_dd=dd))
            in_pos = False
            entry_px = None
            entry_idx = None
            open_date = None

        # Actualizar equity
        if in_pos:
            equity.iloc[i] = equity.iloc[i-1] * (close.iloc[i]/close.iloc[i-1])
        else:
            equity.iloc[i] = equity.iloc[i-1]

    # Si queda posición abierta, cerrarla al final
    if in_pos and entry_idx is not None:
        exit_px = close.iloc[-1]
        trade_slice = close.iloc[entry_idx:]
        peak = trade_slice.cummax()
        dd = (trade_slice/peak - 1.0).min()
        trades.append(Trade(entry_date=open_date, entry_price=entry_px,
                            exit_date=df.index[-1], exit_price=exit_px,
                            ret=exit_px/entry_px - 1.0, max_dd=dd))
    return equity, trades

def metrics_from_equity(equity: pd.Series, periods_per_year=52) -> Dict[str, float]:
    rets = equity.pct_change().fillna(0)
    return {
        'CAGR': CAGR(equity, periods_per_year),
        'Sharpe': sharpe_ratio(rets, periods_per_year),
        'MaxDrawdown': rolling_max_drawdown(equity),
        'TotalReturn': equity.iloc[-1]/equity.iloc[0]-1.0
    }


## Ejecución para múltiples tickers y comparación contra benchmark

In [7]:
# ----- CELDA 7 – Función principal de estrategia 10 con 30 -----

def run_strategy(tickers, start, end, benchmark='SPY'):
    results = {}

    for ticker in tickers:
        try:
            data = yf.download(ticker, start=start, end=end, interval='1wk')
            if data.empty:
                print(f"⚠️ No hay datos para {ticker}")
                continue

            # Lógica de estrategia (cruce SMA10 vs SMA30)
            data['SMA10'] = data['Close'].rolling(10).mean()
            data['SMA30'] = data['Close'].rolling(30).mean()
            data['Position'] = np.where(data['SMA10'] > data['SMA30'], 1, 0)
            data['Returns'] = data['Close'].pct_change()
            data['Strategy'] = data['Returns'] * data['Position'].shift(1)

            # Métricas
            cagr = (1 + data['Strategy']).prod()**(52/len(data)) - 1
            sharpe = data['Strategy'].mean() / data['Strategy'].std() * np.sqrt(52)
            mdd = ((data['Close'].cummax() - data['Close']) / data['Close'].cummax()).max()

            metrics_dict = {
                'CAGR': cagr,
                'Sharpe': sharpe,
                'MaxDD': mdd
            }

            # Benchmark
            bench = yf.download(benchmark, start=start, end=end, interval='1wk')
            bench['Returns'] = bench['Close'].pct_change()
            bench_cagr = (1 + bench['Returns']).prod()**(52/len(bench)) - 1
            bench_sharpe = bench['Returns'].mean() / bench['Returns'].std() * np.sqrt(52)
            bench_mdd = ((bench['Close'].cummax() - bench['Close']) / bench['Close'].cummax()).max()

            bench_dict = {
                'CAGR': bench_cagr,
                'Sharpe': bench_sharpe,
                'MaxDD': bench_mdd
            }

            # Trades (placeholder: lista vacía o DataFrame)
            trades_df = pd.DataFrame(columns=['EntryDate', 'ExitDate', 'Return'])

            results[ticker] = {
                'metrics': metrics_dict,
                'bench_metrics': bench_dict,
                'trades': trades_df
            }

        except Exception as e:
            print(f"Error procesando {ticker}: {e}")
            continue

    return results


## Visualizaciones

In [8]:
#Celda 8
def plot_score(df: pd.DataFrame, ticker: str):
    fig, ax = plt.subplots()
    df['Score'].plot(ax=ax, label='Score')
    ax.axhline(8, linestyle='--', label='Umbral 8')
    ax.set_title(f'Score semanal – {ticker}')
    ax.legend()
    plt.show()

def plot_equity_vs_benchmark(equity: pd.Series, bench_equity: pd.Series, ticker: str):
    fig, ax = plt.subplots()
    equity.plot(ax=ax, label='Estrategia')
    bench_equity.plot(ax=ax, label='Benchmark')
    ax.set_title(f'Equity Curve – {ticker} vs Benchmark')
    ax.legend()
    plt.show()

def plot_base_breakout(df: pd.DataFrame, ticker: str, window: int = 60):
    """Ejemplo de base detectada y ruptura. Muestra velas simples, WMA30 y techo de base."""
    from matplotlib.patches import Rectangle

    # Buscar una semana con breakout
    idxs = df.index[df['Breakout'] == 1]
    if len(idxs) == 0:
        print("No se encontró un breakout en el período.")
        return
    i = df.index.get_loc(idxs[-1])  # último breakout
    i0 = max(0, i - window)
    d = df.iloc[i0:i+1]

    # Velas
    fig, ax = plt.subplots(figsize=(12,6))
    for j, (date, row) in enumerate(d.iterrows()):
        x = j
        o, h, l, c = row['Open'], row['High'], row['Low'], row['Close']
        # cuerpo
        lower, height = (min(o,c), abs(c-o))
        ax.add_patch(Rectangle((x-0.3, lower), 0.6, height))
        # mechas
        ax.vlines(x, l, h)

    ax.plot(range(len(d)), d['WMA30'], label='WMA30')
    ax.hlines(d['BaseHigh'].iloc[-1], xmin=0, xmax=len(d)-1, linestyles='--', label='Techo base')

    ax.set_xticks(range(0, len(d), max(1, len(d)//8)))
    ax.set_xticklabels([d.index[k].strftime('%Y-%m-%d') for k in range(0, len(d), max(1, len(d)//8))], rotation=45)
    ax.set_title(f'{ticker} – Base y ruptura')
    ax.legend()
    plt.tight_layout()
    plt.show()


## Ejecutar con tus tickers

#El objetivo del backtest es medir:

CAGR_Strategy → crecimiento anual compuesto si hubieras seguido la estrategia. Bueno si es mayor al benchmark (SPY u otro índice).

Ejemplo: Estrategia CAGR = 18%. Benchmark CAGR = 10% → Tu sistema le gana al mercado.

Sharpe_Strategy → ratio riesgo/retorno. Regla práctica: Sharpe < 1 → pobre. 1 ≤ Sharpe < 2 → aceptable. 2 ≤ Sharpe < 3 → muy bueno. 3 → excelente

MaxDD_Strategy → drawdown máximo (la peor caída desde un pico). Bueno si es menor al benchmark.

TotalReturn_Strategy → retorno total.

N_Trades → cuántas operaciones ejecutó la estrategia.

👉 Si N_Trades = 0, significa que las reglas nunca se activaron. Entonces esa estrategia no puede evaluarse porque no generó operaciones.

# ¿Cómo interpretarlo?

Si un ticker muestra CAGR_Strategy > CAGR_Bench, la estrategia logró “ganar al mercado”.

Un Sharpe > 1 ya es aceptable, > 2 excelente.

MaxDD bajo (ej. < -20%) indica mejor control de pérdidas.

N_Trades bajo pero con buena CAGR = sistema selectivo pero potente.

En gráficos, confirma que el score subió a ≥8 en bases sólidas y que la salida se dio al deteriorarse la WMA30 (fase 3).

Equity Curve vs Benchmark: La curva del capital (estrategia) debería ser más suave, consistente y ascendente que la del benchmark.Si tu curva es muy parecida al benchmark pero con más volatilidad → la estrategia no aporta valor.

In [12]:
# ----- CELDA 9 – Evaluación de tickers -----
from datetime import date

tickers = ['MEDP', 'TBBK', 'PODD', 'HG', 'HNGE', 'AMSC', 'KNSA', 'ALAB', 'ALNT', 'FNV', 'PRM', 'ARX', 'STX', 'RMBS', 'KGC', 'URBN', 'BAP', 'EGO', 'WDC', 'HOOD', 'APH', 'RKLB', 'EXTR', 'NVDA', 'TFPM', 'ANET', 'ATAT', 'FIX', 'OUST', 'WGS', 'LMND', 'LIF', 'RDDT', 'ARQT', 'TARS', 'MIRM', 'HSAI', 'ONC', 'KRMN']
benchmark = 'SPY'
start = '2015-01-01'  # Ajusta si el ticker es más reciente
end = date.today().strftime('%Y-%m-%d') # Define 'end' using today's date

print(f"🔍 Evaluando {len(tickers)} tickers...")

results = run_strategy(tickers, benchmark=benchmark, start=start, end=end)

# Ver qué tickers sí se procesaron
procesados = list(results.keys())
no_procesados = [t for t in tickers if t not in procesados]
print(f"✅ Procesados correctamente: {procesados}")

if no_procesados:
    print(f"⚠️ Ignorados (sin datos o inválidos): {no_procesados}")

if not results:
    print("⚠️ No se obtuvieron resultados para ninguno de los tickers. Revisa mensajes previos.")
else:
    # Mostrar métricas resumidas
    rows = []
    for t, r in results.items():
        m = r.get('metrics', {})
        mb = r.get('bench_metrics', {})
        trades = r.get('trades', [])

        if m and mb:  # solo agregamos si hay métricas válidas
            rows.append({
                'Ticker': t,
                'CAGR_Strategy': m.get('CAGR', np.nan),
                'Sharpe_Strategy': m.get('Sharpe', np.nan),
                'MaxDD_Strategy': m.get('MaxDrawdown', np.nan),
                'TotalReturn_Strategy': m.get('TotalReturn', np.nan),
                'CAGR_Bench': mb.get('CAGR', np.nan),
                'Sharpe_Bench': mb.get('Sharpe', np.nan),
                'MaxDD_Bench': mb.get('MaxDrawdown', np.nan),
                'TotalReturn_Bench': mb.get('TotalReturn', np.nan),
                'N_Trades': len(trades)
            })
        else:
            print(f"⚠️ {t}: no se calcularon métricas (probablemente pocos datos o error en el backtest).")

    if rows:
        summary = pd.DataFrame(rows).set_index('Ticker').sort_index()
        display(summary)
    else:
        print("⚠️ Ningún ticker generó métricas para mostrar.")

/tmp/ipython-input-2248054234.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, interval='1wk')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2248054234.py:32: FutureWarning: YF.download() has changed argument auto_adjust default to True
  bench = yf.download(benchmark, start=start, end=end, interval='1wk')
[*********************100%***********************]  1 of 1 completed

🔍 Evaluando 39 tickers...



/tmp/ipython-input-2248054234.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, interval='1wk')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2248054234.py:32: FutureWarning: YF.download() has changed argument auto_adjust default to True
  bench = yf.download(benchmark, start=start, end=end, interval='1wk')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2248054234.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, interval='1wk')
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-2248054234.py:32: FutureWarning: YF.download() has changed argument auto_adjust default to True
  bench = yf.download(benchmark, start=start, end=end, interval='1wk')
[*********************100%***********************]  1 of 1

✅ Procesados correctamente: ['MEDP', 'TBBK', 'PODD', 'HG', 'HNGE', 'AMSC', 'KNSA', 'ALAB', 'ALNT', 'FNV', 'PRM', 'ARX', 'STX', 'RMBS', 'KGC', 'URBN', 'BAP', 'EGO', 'WDC', 'HOOD', 'APH', 'RKLB', 'EXTR', 'NVDA', 'TFPM', 'ANET', 'ATAT', 'FIX', 'OUST', 'WGS', 'LMND', 'LIF', 'RDDT', 'ARQT', 'TARS', 'MIRM', 'HSAI', 'ONC', 'KRMN']


,CAGR_Strategy,Sharpe_Strategy,MaxDD_Strategy,TotalReturn_Strategy,CAGR_Bench,Sharpe_Bench,MaxDD_Bench,TotalReturn_Bench,N_Trades
Ticker,,,,,,,,,
ALAB,0.0620,0.3571,NaN,NaN,0.1332,0.8699,NaN,NaN,0
ALNT,-0.0199,0.1289,NaN,NaN,0.1332,0.8699,NaN,NaN,0
AMSC,0.2127,0.6085,NaN,NaN,0.1332,0.8699,NaN,NaN,0
ANET,0.2580,0.8468,NaN,NaN,0.1332,0.8699,NaN,NaN,0
APH,0.0959,0.5812,NaN,NaN,0.1332,0.8699,NaN,NaN,0
ARQT,0.0087,0.2346,NaN,NaN,0.1332,0.8699,NaN,NaN,0
ARX,0.0000,NaN,NaN,NaN,0.1332,0.8699,NaN,NaN,0
ATAT,0.0072,0.1540,NaN,NaN,0.1332,0.8699,NaN,NaN,0
BAP,0.0289,0.2420,NaN,NaN,0.1332,0.8699,NaN,NaN,0


In [10]:
#Celda 10
# Elegir el primer ticker con resultados y graficar
for t in tickers:
    if t in results and 'df' in results[t]:
        df = results[t]['df']
        plot_score(df, t)
        plot_equity_vs_benchmark(results[t]['equity'], results[t]['bench_equity'], t)
        plot_base_breakout(df, t, window=60)
        break



### Notas y supuestos
- **RS**: se usa *outperformance* de 26 semanas (6 meses) como `RS = logret(asset) - logret(benchmark)`; RS>0 indica que el activo superó al benchmark en ese horizonte.
- **Pendiente WMA30**: plana si está entre -0.1% y +0.1% semanal.
- **Base**: se busca una ventana que termine en la semana actual con duración ≥5 y **volatilidad** `(max-min)/mid` ≤ 10%. La mejor ventana encontrada define `BaseLen`, `BaseVol` y `BaseHigh`.
- **Resistencias**: se estiman con *swing highs* simples. Si el precio está dentro de 5% del **ATH de 10 años**, se considera "cerca de ATH".
- **Volumen**:
  - 2 puntos si el promedio del último cuarto de la base > 20% del promedio de la primera mitad (expansión hacia el final).
  - 1 punto si el promedio de la base < promedio de 52 semanas (volumen seco/decreciente).
- **Breakout**: `Close > BaseHigh` con volumen ≥ 2× el promedio de 10 semanas.
- **Entrada**: cruce de `Score` desde <8 a ≥8 **y** `Breakout==1` esa semana.
- **Salida**: cuando la pendiente de WMA30 < -0.1% semanal (inicio Fase 3).
- La lógica es modular para que ajustes umbrales/ventanas a tu estilo.
